***
# ISOM3360 Data Mining for Business Analytics
## Group 23 Project Code - Credit Card Defaultee Analysis
### Part 1 - Data Preprocessing
***

Name: LAM, Ho Chit  
ITSC:   
SID: 

Name: LEE, Ho Wan Owen  
ITSC: hwolee  
SID: 20604852

Name: LEE, Wai Chung  
ITSC:   
SID: 

### Preface

Credit card blah blah blah

In this Jupyter notebook, extensive data preprocessing will be performed, along with detailed documentation of each step.

In [ ]:
# import library used for data management
import numpy as np 
import pandas as pd 

#### Data Exploration

In [ ]:
# load datasets
path = "./data_raw/"
df_raw = pd.read_csv(path + 'train.csv', index_col='ID')

In [ ]:
# view first 5 rows in train data frame
df_raw.head()

In [ ]:
# get train set info
df_raw.info()

# Data Preparation - Deal with missing values

In [ ]:
# create a copy of train set
train_df = train.copy()

In [ ]:
# replace missing value for "Age" in training set by using mean of 'Age' from train data
train_df['Age'].fillna(train['Age'].mean(),inplace = True)

In [ ]:
# check whether missing value in "Age" is replaced or not
train_df['Age'].describe()

In [ ]:
# replace missing value for "Embarked" in train data with mode
train_df['Embarked'].fillna(train['Embarked'].mode()[0],inplace = True)

In [ ]:
train_df['Embarked'].describe()

In [ ]:
# Alternatively, you can also drop the two rows with missing value for 'Emabarked'
#train_df.dropna(subset=['Embarked'],how='any',inplace=True)

In [ ]:
# drop 'Cabin' for both train and test data
train_df = train_df.drop(columns='Cabin')

In [ ]:
train_df.info()

In [ ]:
train_df['Survived'].value_counts(normalize=True)

How will you fill missing value for test set?

In [ ]:
# find out total number of missing values 
test.isna().sum()

In [ ]:
# creat a copy of test set
test_df = test.copy()

In [ ]:
# replacing missing vale for 'Age' in test set, is the following code correct?
test_df['Age'].fillna(test['Age'].mean(),inplace = True)

# Data Preparation - Discretization for Continuous Variables

In [ ]:
# Bining / Descritization
# give names for different age group
group_names = ['Young', 'Middle_aged', 'Senior']

In [ ]:
# divide Age into 3 equal interval groups and give corresponding names
train_df['Age-binned']=pd.cut(train_df['Age'], 3 , labels=group_names)

In [ ]:
# View Age-binned in bar chart
train_df['Age-binned'].value_counts().plot(kind='bar')

When do you want to discretize Continuous Variables? Is it always good to have binning?

# Data Preparation - Normalization for Continuous Variables

In [ ]:
# normalize Fare
# import library 
from sklearn import preprocessing

In [ ]:
# create min-max scaler based on Fare from train set
minmax_scaler = preprocessing.MinMaxScaler().fit(train[['Fare']])

In [ ]:
# Apply min-max scaler normalization on a single attribute
train_df['Fare_minmax']=minmax_scaler.transform(train[['Fare']])

In [ ]:
# Apply z-score normalization on a single attribute
zscore_scaler = preprocessing.StandardScaler().fit(train[['Fare']])
train_df['Fare_zscore']=zscore_scaler.transform(train[['Fare']])

What should we do regarding the 'Fare' in test set?

In [ ]:
# Apply min-max scaler in test set, is the following code correct?
zscore_scaler_test = preprocessing.StandardScaler().fit(test[['Fare']])
test_df['Fare_zscore']=zscore_scaler_test.transform(test[['Fare']])

Can we drop the row with missing value for 'Fare' in test set?

# Data Preparation - Handle Categorical Variables

In [ ]:
# get dummy variables for categorical varialbes 'Sex' and 'Embarked' in train 
train_df_dummy =pd.get_dummies(train, columns=['Sex', 'Embarked'])

# this line of code not only help you to get dummies and autometically drop the first column, it also help you to delete the original column

In [ ]:
train_df_dummy

In [ ]:
train_df =pd.get_dummies(train_df, columns=['Sex', 'Embarked'],drop_first = True)

What changes we need to do for 'Sex' and 'Embarked' in test set?

In [ ]:
train_df.head()

In [ ]:
# save the datasets and write to csv, so you don't need to do the data cleaning from the very beginning
train_df.to_csv('titanic_cleaned')

In [ ]:
train_df.info()

## Let's build our first Decision Tree
we will use all the train data (891 examples) to construct the tree and evaluate the model

### Step 1: Define features and target varialbe

In [ ]:
# define independent variables / attirbutes / features
features = ['Pclass','Age','SibSp','Parch','Fare','Sex_male','Embarked_Q','Embarked_S']
# define one single target variable / label
target = ['Survived']

In [ ]:
# get defined training dataset
X = train_df[features]
y = train_df[target]

In [ ]:
X.info()

### Step 2: Split data into training and testing set

In [ ]:
# import train split function
from sklearn.model_selection import train_test_split

In [ ]:
# split data into 60% and 40%, put 40% in testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state = 23)

### Step 3: Build a Tree based on 60% train data

In [ ]:
# import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# define our model by using the default value
model = DecisionTreeClassifier(random_state=42)

In [ ]:
# train the model by fit in 60% training data
model.fit(X_train, y_train)

### Visualize the Decision Tree

In [ ]:
# import library for visuaiization
from sklearn import tree
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize = (100,150))
tree.plot_tree(model,ax=None, fontsize=50)
plt.show()

In [ ]:
from sklearn.tree import export_graphviz
from IPython.display import Image
import pydotplus
import graphviz

In [ ]:
dot_data = tree.export_graphviz(model, out_file=None, 
                      feature_names=features,  
                      class_names=['Did not survive', 'Survived'],
                      filled = True, rounded=True,  
                      special_characters=True)

In [ ]:
graph = graphviz.Source(dot_data)

In [ ]:
graph

In [ ]:
graph.render("titanic_decitiontree")

### Step 4: Evaluate the model on 40% hold out test set

In [ ]:
# import evaluation functions from sklearn, let's just focus on Accuracy for now
from sklearn.metrics import accuracy_score

In [ ]:
# get prediction for X_test
pred_test = model.predict(X_test)

In [ ]:
# evaluate result 
print("Accuracy:",accuracy_score(y_test, pred_test, normalize=True, sample_weight=None))

How do you evaluate this accuracy rate? Is it acceptable? Is it realiable?

Compute confusion matrix

Calculate precision and recall

Calculate F1 score

Give ROC curve

GridSearchCV

Cross validation